In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import escalateclient
import importlib

First, login to the escalate server where your data is hosted. In this case we are logging in to a local instance of the webserver running on port 8000

In [2]:
importlib.reload(escalateclient)
server_url = 'http://localhost:8000'
username = 'vshekar'
password = 'copperhead123'
client = escalateclient.ESCALATEClient(server_url, username, password)

In [6]:
# Get the Nielson lab
nielson_lab_response = client.get(endpoint='actor', data={'description': 'Nielson Lab', 'person__isnull': True, 'systemtool__isnull': True})

GET: OK
Found one resource, returning dict
